In [24]:
import numpy as np
import pandas as pd
from tqdm import tqdm

from sklearn.ensemble import RandomForestClassifier 

import warnings
from multiprocessing import cpu_count

import os
import bottleneck as bn
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import seaborn as sns
from joblib import Parallel, delayed
from scipy import signal
from scipy.special import gamma, psi
from sklearn.base import BaseEstimator
from sklearn.decomposition import PCA
from sklearn.feature_selection import RFECV, SelectFromModel
from sklearn.feature_selection._base import SelectorMixin
from sklearn.linear_model import LogisticRegression
from sklearn.neighbors import NearestNeighbors
from sklearn.preprocessing import StandardScaler
from sklearn.svm import SVC, SVR
from sklearn.naive_bayes import GaussianNB, BernoulliNB, ComplementNB
from sklearn.utils import check_X_y
from sklearn.discriminant_analysis import QuadraticDiscriminantAnalysis as QDA
from sklearn.ensemble import GradientBoostingClassifier, AdaBoostClassifier, BaggingClassifier, RandomForestClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.neural_network import MLPClassifier
from sklearn.ensemble import StackingClassifier
from sklearn.preprocessing import FunctionTransformer
from sklearn.ensemble import VotingClassifier
from tqdm import tqdm


from src.train import cv, calculate_score, prepare_cv_indices
from src.feature_selector import BaseFeatureSelector
from src.custom_feature_selectors.mi import MIFeatureSelector, CMIM, JMIM, IGFS
from src.custom_feature_selectors.boruta import Boruta
from src.custom_feature_selectors.feature_importance import RandomForestFeatureImportanceSelector
from src.settings import DATA_DIR
from src.experiment_utils import perform_experiments, find_best_experiments
from src.experiment import Experiment

warnings.filterwarnings('ignore') 

In [25]:
# Read the text file into a dataframe
X = pd.read_csv(os.path.join(DATA_DIR, 'x_train.txt'), sep=' ', header=None).to_numpy()
y = pd.read_csv(os.path.join(DATA_DIR, 'y_train.txt'), header=None).to_numpy().T[0]
X_test = pd.read_csv(os.path.join(DATA_DIR, 'x_test.txt'), sep=' ', header=None).to_numpy()

# Read the shape of the data
print(X.shape, y.shape)
print(X_test.shape)

(5000, 500) (5000,)
(5000, 500)


In [26]:
X_train = X.copy()
X_train_orig = X.copy()

transformer = FunctionTransformer(np.log1p)
X_train = np.concatenate(
    [X_train, transformer.fit_transform(X_train_orig)], axis=1
)

transformer2 = FunctionTransformer(np.sqrt)
X_train = np.concatenate(
    [X_train, transformer2.fit_transform(X_train_orig)], axis=1
)

transformer3 = FunctionTransformer(np.exp)
X_train = np.concatenate(
    [X_train, transformer3.fit_transform(X_train_orig)], axis=1
)

if np.isnan(X_train).any():
    # if so, replace NaNs with 0
    X_train = np.nan_to_num(X_train)

In [27]:
# from sklearn import tree


# class AdaBoost():
    
#     def __init__(self, max_depth=1):
#         self.max_depth = max_depth
#         self.models = []
#         self.betas = []
#         self.weights = None
#         self.n_classes = None
#         self._stopped = False
    
#     def fit(self, X, y, n_iter=100):
#         n = X.shape[0]
#         if self.weights is None:
#             self.weights = np.ones(n) / n
#             self.n_classes = len(np.unique(y))
        
#         for i in range(n_iter):
#             model = tree.DecisionTreeClassifier(max_depth=self.max_depth)
#             model.fit(X, y, sample_weight = self.weights)
#             y_pred = model.predict(X)
#             err = self.calculate_error(y, y_pred)
#             if err < 1e-14:
#                 self._stopped = True
#                 break

#             self.models.append(model)
#             self.betas.append(err / (1 - err))
#             self.update_weights(y, y_pred)
            
#     def predict(self, X):
#         n_models = len(self.models)
#         class_preds = np.zeros((X.shape[0], self.n_classes))
        
#         for i in range(n_models):
#             y_pred = self.models[i].predict(X)
#             for j in range(self.n_classes):
#                 y_weighted_pred = (y_pred == np.ones(y_pred.shape[0]) * j) * np.log(1 / self.betas[i])
#                 class_preds[:, j] = class_preds[:, j] + y_weighted_pred
                
#         return np.argmax(class_preds, axis=1)
        
#     def calculate_error(self, y_true, y_pred):
#         return np.sum((y_true != y_pred) * self.weights)
    
#     def update_weights(self, y_true, y_pred):
#         scaler = np.ones(self.weights.shape[0])
#         scaler[y_true == y_pred] = self.betas[-1]
#         self.weights = self.weights * scaler
#         sum_weights = np.sum(self.weights)
#         self.weights /= sum_weights

#     def predict_proba(self, X):
#         n_models = len(self.models)
#         class_preds = np.zeros((X.shape[0], self.n_classes))
        
#         for i in range(n_models):
#             y_pred = self.models[i].predict(X)
#             for j in range(self.n_classes):
#                 y_weighted_pred = (y_pred == np.ones(y_pred.shape[0]) * j) * np.log(1 / self.betas[i])
#                 class_preds[:, j] = class_preds[:, j] + y_weighted_pred
                
#         return class_preds / np.sum(class_preds, axis=1).reshape(-1, 1)
        
        

In [28]:
from sklearn import svm


n_features = 3

experiment_config = [
    
    # Experiment(
    #     classifier=GradientBoostingClassifier,
    #     classifier_config = {
    #         "n_estimators":600, 
    #         "learning_rate":1.2,
    #         "max_depth":1,
    #         "n_iter_no_change": 10,
    #         "min_samples_split": 20,

    #     },
    #     feature_selector=RandomForestFeatureImportanceSelector,
    #     feature_selector_config={
    #         "n_features": n_features,
    #     }
    # ),

    # Experiment(
    #     classifier=AdaBoostClassifier,
    #     classifier_config = {
    #         "n_estimators": 300,
    #         "learning_rate": 0.1,
    #     },
    #     feature_selector=RandomForestFeatureImportanceSelector,
    #     feature_selector_config={
    #         "n_features": n_features,
    #     }
    # ),

    # Experiment(
    #     classifier=MLPClassifier,
    #     classifier_config = {'activation': 'relu', 
    #                          'alpha': 1e-05, 
    #                          'hidden_layer_sizes': (100, 100, 50,), 
    #                          'learning_rate': 'constant', 
    #                          'learning_rate_init': 0.001, 
    #                          'solver': 'sgd'
    #                          },
    #     feature_selector=RandomForestFeatureImportanceSelector,
    #     feature_selector_config={
    #         "n_features": n_features,
    #     }
    # ),

    # Experiment(
    #     classifier=StackingClassifier,
    #     classifier_config = {'estimators': [
    #         ('gnb', GaussianNB()),
    #         ('qda', QDA()),
    #         ('ada', AdaBoostClassifier(n_estimators=100)),
    #         ],
    #         'final_estimator': LogisticRegression(penalty='l2', C=1, solver='lbfgs', max_iter=1000),
    #     },
    #     feature_selector=RandomForestFeatureImportanceSelector,
    #     feature_selector_config={
    #         "n_features": n_features,
    #     },
    #     # transformer=True,
    # ),
    Experiment(
        classifier=VotingClassifier,
        classifier_config = {'estimators': [
            ('gnb', GaussianNB()),
            ('qda', QDA()),
            ('ada', AdaBoostClassifier(n_estimators=100)),
            ],
            'voting': 'soft',
        },
        feature_selector=RandomForestFeatureImportanceSelector,
        feature_selector_config={
            "n_features": n_features,
        },
        # transformer=True,
    ),
    
]

In [29]:
scores, indices = perform_experiments(
    X[:, [0, 1, 100, 102, 103]], 
    y, experiment_config)
# X_train[:, [8, 100,  101,  102, 103, 104, 105, 605, 1100, 1508, 1600, 1601, 1602, 1603, 1604, 1605]

Experiment exp_vc_rffis_a20427 in progress...
Using 3 features, we properly classified 150/200 clients.
Using 3 features, we properly classified 151/200 clients.
Using 3 features, we properly classified 151/200 clients.
Using 3 features, we properly classified 152/200 clients.
Using 3 features, we properly classified 155/200 clients.


In [13]:
indices

{'exp_sc_jmim_f47b41': [array([0, 1, 2]),
  array([0, 1, 2]),
  array([0, 1, 2]),
  array([0, 1, 2]),
  array([0, 1, 2])]}

In [7]:
unique_indices = set()

# Iterate over all arrays in the dictionary and add their elements to the set
for key in indices:
    for array in indices[key]:
        unique_indices.update(array)

# Convert the set back to a sorted list
unique_indices = sorted(list(unique_indices))

print(unique_indices)

[2, 3, 4]


In [101]:
# small_best_indices = []
# for id in [0, 1, 2, 3, 4, 7, 8, 9, 11, 15, 16, 17, 27, 28, 29]:
#     small_best_indices.append(best_indices[id])
# small_best_indices
small_best_indices = [0, 1, 2, 3, 4, 7, 29, 34, 40, 100, 101, 102, 103, 105, 144, 340, 497]

In [3]:
best_indices = [0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 13, 16, 17, 29, 34, 40, 45, 48, 74, 100, 101, 102, 103, 104, 105, 144, 340, 497]

X[:, best_indices].shape

(5000, 30)

In [7]:
from sklearn.discriminant_analysis import QuadraticDiscriminantAnalysis
# model = RandomForestClassifier(max_depth=5)
model = QuadraticDiscriminantAnalysis()
k_folds = 3

scores = cv(X, y, model, k_folds)
scores

Using 500 features, we properly classified 176/333 clients.
Using 500 features, we properly classified 187/333 clients.
Using 500 features, we properly classified 165/333 clients.


[-94721.05578884223, -94391.12177564488, -95048.01920768307]

In [15]:
model = RandomForestClassifier(max_depth=5)
feature_selector = MIFS(n_features_to_select=10)
k_folds = 5

scores = cv(X, y, model, k_folds, feature_selector=feature_selector)
scores

  0%|          | 0/500 [00:00<?, ?it/s]

  5%|▌         | 27/500 [01:14<21:38,  2.74s/it]


KeyboardInterrupt: 

In [14]:


class MIFS(BaseFeatureSelector):

    def __init__(self, n_features_to_select, n_neighbors=3):
        super().__init__(n_features_to_select)
        self.n_neighbors = n_neighbors

    def fit(self, X, y):
        self.n_features_ = X.shape[1]
        self.scores_ = np.zeros(self.n_features_)

        for i in tqdm(range(self.n_features_)):
            s = self._mifs(X, y, i)
            self.scores_[i] = s

        self.ranks_ = np.argsort(self.scores_)[::-1]
        self.support_ = np.zeros(self.n_features_, dtype=bool)
        self.support_[self.ranks_[: self.n_features_to_select]] = True

    def transform(self, X: np.ndarray) -> np.ndarray:
        return X[:, self.support_]

    def _mifs(self, X, y, i):
        n_samples, n_features = X.shape
        x_i = X[:, i]
        x_i = x_i.reshape(-1, 1)

        scores = np.zeros(n_features)
        for j in range(n_features):
            if j == i:
                continue

            x_j = X[:, j]
            x_j = x_j.reshape(-1, 1)

            x_ij = np.concatenate([x_i, x_j], axis=1)

            knn = NearestNeighbors(n_neighbors=self.n_neighbors)
            knn.fit(x_ij)
            d_ij, _ = knn.kneighbors(x_ij)

            d_i = d_ij[:, 0]
            d_j = d_ij[:, 1]

            mi = self._mi(d_i, d_j)
            scores[j] = mi

        return np.mean(scores)

    def _mi(self, d_i, d_j):
        n_samples = len(d_i)
        n_bins = int(np.sqrt(n_samples))

        hist_2d, _, _ = np.histogram2d(d_i, d_j, bins=n_bins)
        p_ij = hist_2d / n_samples

        p_i = np.sum(p_ij, axis=1)
        p_j = np.sum(p_ij, axis=0)

        mi = np.sum(p_ij * np.log(p_ij / np.outer(p_i, p_j) + 1e-10))

        return mi

In [11]:
class MIFS2(FeatureSelectorWrapper):

    def __init__(self):
        super().__init__(MutualInformationFeatureSelector())

    def transform(self, X: np.ndarray) -> np.ndarray:
        support = self.model._support_mask
        return X[:, support]
    
model = RandomForestClassifier(max_depth=5)
model = QuadraticDiscriminantAnalysis()
feature_selector = MIFS2()
k_folds = 5

scores = cv(X, y, model, k_folds, feature_selector=feature_selector)
scores

Using 13 features, we properly classified 158/200 clients.
Using 12 features, we properly classified 152/200 clients.
Using 17 features, we properly classified 133/200 clients.
Using 17 features, we properly classified 126/200 clients.
Using 14 features, we properly classified 159/200 clients.


[5300.0, 5200.0, 3250.0, 2900.0, 5150.0]

In [10]:
class MIFS2(BaseFeatureSelector):

    def __init__(self):
        self.model = MutualInformationFeatureSelector()

    def fit(self, X, y):
        self.model.fit(X, y)

    def transform(self, X: np.ndarray) -> np.ndarray:
        support = self.model._support_mask
        return X[:, support]
    

# model = RandomForestClassifier(max_depth=5)
model = LogisticRegression()
feature_selector = MIFS2()
k_folds = 5

scores = cv(X, y, model, k_folds, feature_selector=feature_selector)
scores

Using 19 features, we properly classified 103/200 clients.


KeyboardInterrupt: 

In [9]:
# Copyright (c) 2016, Daniel Homola
# All rights reserved.

# Redistribution and use in source and binary forms, with or without
# modification, are permitted provided that the following conditions are met:

# * Redistributions of source code must retain the above copyright notice, this
#   list of conditions and the following disclaimer.

# * Redistributions in binary form must reproduce the above copyright notice,
#   this list of conditions and the following disclaimer in the documentation
#   and/or other materials provided with the distribution.

# * Neither the name of mifs nor the names of its
#   contributors may be used to endorse or promote products derived from
#   this software without specific prior written permission.

# THIS SOFTWARE IS PROVIDED BY THE COPYRIGHT HOLDERS AND CONTRIBUTORS "AS IS"
# AND ANY EXPRESS OR IMPLIED WARRANTIES, INCLUDING, BUT NOT LIMITED TO, THE
# IMPLIED WARRANTIES OF MERCHANTABILITY AND FITNESS FOR A PARTICULAR PURPOSE ARE
# DISCLAIMED. IN NO EVENT SHALL THE COPYRIGHT HOLDER OR CONTRIBUTORS BE LIABLE
# FOR ANY DIRECT, INDIRECT, INCIDENTAL, SPECIAL, EXEMPLARY, OR CONSEQUENTIAL
# DAMAGES (INCLUDING, BUT NOT LIMITED TO, PROCUREMENT OF SUBSTITUTE GOODS OR
# SERVICES; LOSS OF USE, DATA, OR PROFITS; OR BUSINESS INTERRUPTION) HOWEVER
# CAUSED AND ON ANY THEORY OF LIABILITY, WHETHER IN CONTRACT, STRICT LIABILITY,
# OR TORT (INCLUDING NEGLIGENCE OR OTHERWISE) ARISING IN ANY WAY OUT OF THE USE
# OF THIS SOFTWARE, EVEN IF ADVISED OF THE POSSIBILITY OF SUCH DAMAGE.

NUM_CORES = cpu_count()

def get_mi_vector(MI_FS, F, s):
    """
    Calculates the Mututal Information between each feature in F and s.

    This function is for when |S| > 1. s is the previously selected feature.
    We exploite the fact that this step is embarrassingly parallel.
    """

    MIs = Parallel(n_jobs=MI_FS.n_jobs)(delayed(_get_mi)(f, s, MI_FS)
                                        for f in F)
    return MIs


def _get_mi(f, s, MI_FS):
    
    n, p = MI_FS.X.shape
    if MI_FS.method in ['JMI', 'JMIM']:
        # JMI & JMIM
        joint = MI_FS.X[:, (s, f)]
        if MI_FS.categorical:
            MI = _mi_dc(joint, MI_FS.y, MI_FS.k)
        else:
            vars = (joint, MI_FS.y)
            MI = _mi_cc(vars, MI_FS.k)
    else:
        # MRMR
        vars = (MI_FS.X[:, s].reshape(n, 1), MI_FS.X[:, f].reshape(n, 1))
        MI = _mi_cc(vars, MI_FS.k)

    # MI must be non-negative
    if MI > 0:
        return MI
    else:
        return np.nan


def get_first_mi_vector(MI_FS, k):
    """
    Calculates the Mututal Information between each feature in X and y.

    This function is for when |S| = 0. We select the first feautre in S.
    """
    n, p = MI_FS.X.shape
    MIs = Parallel(n_jobs=MI_FS.n_jobs)(delayed(_get_first_mi)(i, k, MI_FS)
                                        for i in range(p))
    return MIs


def _get_first_mi(i, k, MI_FS):
    n, p = MI_FS.X.shape
    if MI_FS.categorical:
        x = MI_FS.X[:, i].reshape((n, 1))
        MI = _mi_dc(x, MI_FS.y, k)
    else:
        vars = (MI_FS.X[:, i].reshape((n, 1)), MI_FS.y)
        MI = _mi_cc(vars, k)

    # MI must be non-negative
    if MI > 0:
        return MI
    else:
        return np.nan


def _mi_dc(x, y, k):
    """
    Calculates the mututal information between a continuous vector x and a
    disrete class vector y.

    This implementation can calculate the MI between the joint distribution of
    one or more continuous variables (X[:, 1:3]) with a discrete variable (y).

    Thanks to Adam Pocock, the author of the FEAST package for the idea.

    Brian C. Ross, 2014, PLOS ONE
    Mutual Information between Discrete and Continuous Data Sets
    """

    y = y.flatten()
    n = x.shape[0]
    classes = np.unique(y)
    knn = NearestNeighbors(n_neighbors=k)
    # distance to kth in-class neighbour
    d2k = np.empty(n)
    # number of points within each point's class
    Nx = []
    for yi in y:
        Nx.append(np.sum(y == yi))

    # find the distance of the kth in-class point
    for c in classes:
        mask = np.where(y == c)[0]
        knn.fit(x[mask, :])
        d2k[mask] = knn.kneighbors()[0][:, -1]

    # find the number of points within the distance of the kth in-class point
    knn.fit(x)
    m = knn.radius_neighbors(radius=d2k, return_distance=False)
    m = [i.shape[0] for i in m]

    # calculate MI based on Equation 2 in Ross 2014
    MI = psi(n) - np.mean(psi(Nx)) + psi(k) - np.mean(psi(m))
    return MI


def _mi_cc(variables, k=1):
    """
    Returns the mutual information between any number of variables.

    Here it is used to estimate MI between continuous X(s) and y.
    Written by Gael Varoquaux:
    https://gist.github.com/GaelVaroquaux/ead9898bd3c973c40429
    """

    all_vars = np.hstack(variables)
    return (sum([_entropy(X, k=k) for X in variables]) -
            _entropy(all_vars, k=k))


def _nearest_distances(X, k=1):
    """
    Returns the distance to the kth nearest neighbor for every point in X
    """

    knn = NearestNeighbors(n_neighbors=k, metric='chebyshev')
    knn.fit(X)
    # the first nearest neighbor is itself
    d, _ = knn.kneighbors(X)
    # returns the distance to the kth nearest neighbor
    return d[:, -1]


def _entropy(X, k=1):
    """
    Returns the entropy of the X.

    Written by Gael Varoquaux:
    https://gist.github.com/GaelVaroquaux/ead9898bd3c973c40429

    Parameters
    ----------
    X : array-like, shape (n_samples, n_features)
        The data the entropy of which is computed
    k : int, optional
        number of nearest neighbors for density estimation

    References
    ----------
    Kozachenko, L. F. & Leonenko, N. N. 1987 Sample estimate of entropy
    of a random vector. Probl. Inf. Transm. 23, 95-101.
    See also: Evans, D. 2008 A computationally efficient estimator for
    mutual information, Proc. R. Soc. A 464 (2093), 1203-1215.
    and:

    Kraskov A, Stogbauer H, Grassberger P. (2004). Estimating mutual
    information. Phys Rev E 69(6 Pt 2):066138.

    F. Perez-Cruz, (2008). Estimation of Information Theoretic Measures
    for Continuous Random Variables. Advances in Neural Information
    Processing Systems 21 (NIPS). Vancouver (Canada), December.
    return d*mean(log(r))+log(volume_unit_ball)+log(n-1)-log(k)

    """

    # Distance to kth nearest neighbor
    r = _nearest_distances(X, k)
    n, d = X.shape
    volume_unit_ball = (np.pi ** (.5 * d)) / gamma(.5 * d + 1)
    return (d * np.mean(np.log(r + np.finfo(X.dtype).eps)) +
            np.log(volume_unit_ball) + psi(n) - psi(k))

class MutualInformationFeatureSelector(BaseEstimator, SelectorMixin):
    """
    MI_FS stands for Mutual Information based Feature Selection.
    This class contains routines for selecting features using both
    continuous and discrete y variables. Three selection algorithms are
    implemented: JMI, JMIM and MRMR.

    This implementation tries to mimic the scikit-learn interface, so use fit,
    transform or fit_transform, to run the feature selection.

    Parameters
    ----------

    method : string, default = 'JMI'
        Which mutual information based feature selection method to use:
        - 'JMI' : Joint Mutual Information [1]
        - 'JMIM' : Joint Mutual Information Maximisation [2]
        - 'MRMR' : Max-Relevance Min-Redundancy [3]

    k : int, default = 5
        Sets the number of samples to use for the kernel density estimation
        with the kNN method. Kraskov et al. recommend a small integer between
        3 and 10.

    n_features : int or string, default = 'auto'
        If int, it sets the number of features that has to be selected from X.
        If 'auto' this is determined automatically based on the amount of
        mutual information the previously selected features share with y.

    categorical : Boolean, default = True
        If True, y is assumed to be a categorical class label. If False, y is
        treated as a continuous. Consequently this parameter determines the
        method of estimation of the MI between the predictors in X and y.

    n_jobs : int, optional (default=1)
        The number of threads to open if possible.

    verbose : int, default=0
        Controls verbosity of output:
        - 0: no output
        - 1: displays selected features
        - 2: displays selected features and mutual information

    Attributes
    ----------

    n_features_ : int
        The number of selected features.

    support_ : array of length X.shape[1]
        The mask array of selected features.

    ranking_ : array of shape n_features
        The feature ranking of the selected features, with the first being
        the first feature selected with largest marginal MI with y, followed by
        the others with decreasing MI.

    mi_ : array of shape n_features
        The JMIM of the selected features. Usually this a monotone decreasing
        array of numbers converging to 0. One can use this to estimate the
        number of features to select. In fact this is what n_features='auto''
        tries to do heuristically.

    Examples
    --------

    import pandas as pd
    import mifs

    # load X and y
    X = pd.read_csv('my_X_table.csv', index_col=0).values
    y = pd.read_csv('my_y_vector.csv', index_col=0).values

    # define MI_FS feature selection method
    feat_selector = mifs.MutualInformationFeatureSelector()

    # find all relevant features
    feat_selector.fit(X, y)

    # check selected features
    feat_selector.support_

    # check ranking of features
    feat_selector.ranking_

    # call transform() on X to filter it down to selected features
    X_filtered = feat_selector.transform(X)

    References
    ----------

    [1] H. Yang and J. Moody, "Data Visualization and Feature Selection: New
        Algorithms for Nongaussian Data"
        NIPS 1999
    [2] Bennasar M., Hicks Y., Setchi R., "Feature selection using Joint Mutual
        Information Maximisation"
        Expert Systems with Applications, Vol. 42, Issue 22, Dec 2015
    [3] H. Peng, Fulmi Long, C. Ding, "Feature selection based on mutual
        information criteria of max-dependency, max-relevance,
        and min-redundancy"
        Pattern Analysis & Machine Intelligence 2005
    """

    def __init__(self, method='JMI', k=5, n_features='auto', categorical=True,
                 n_jobs=1, verbose=0):
        self.method = method
        self.k = k
        self.n_features = n_features
        self.categorical = categorical
        self.n_jobs = n_jobs
        self.verbose = verbose
        self._support_mask = None

    def _get_support_mask(self):
        if self._support_mask is None:
            raise ValueError('mRMR has not been fitted yet!')
        return self._support_mask

    def fit(self, X, y):
        """
        Fits the MI_FS feature selection with the chosen MI_FS method.

        Parameters
        ----------
        X : array-like, shape = [n_samples, n_features]
            The training input samples.

        y : array-like, shape = [n_samples]
            The target values.
        """

        # Check if n_jobs is negative
        if self.n_jobs < 0:
            self.n_jobs = NUM_CORES - self.n_jobs

        self.X, y = self._check_params(X, y)
        n, p = X.shape
        self.y = y.reshape((n, 1))

        # list of selected features
        S = []
        # list of all features
        F = list(range(p))

        if self.n_features != 'auto':
            feature_mi_matrix = np.zeros((self.n_features, p))
        else:
            feature_mi_matrix = np.zeros((n, p))
        feature_mi_matrix[:] = np.nan
        S_mi = []

        # ---------------------------------------------------------------------
        # FIND FIRST FEATURE
        # ---------------------------------------------------------------------

        xy_MI = np.array(get_first_mi_vector(self, self.k))

        # choose the best, add it to S, remove it from F
        S, F = self._add_remove(S, F, bn.nanargmax(xy_MI))
        S_mi.append(bn.nanmax(xy_MI))

        # notify user
        if self.verbose > 0:
            self._print_results(S, S_mi)

        # ---------------------------------------------------------------------
        # FIND SUBSEQUENT FEATURES
        # ---------------------------------------------------------------------
        if self.n_features == 'auto': n_features = np.inf
        else: n_features = self.n_features
         
        while len(S) < n_features:
            # loop through the remaining unselected features and calculate MI
            s = len(S) - 1
            feature_mi_matrix[s, F] = get_mi_vector(self, F, S[-1])

            # make decision based on the chosen FS algorithm
            fmm = feature_mi_matrix[:len(S), F]
            if self.method == 'JMI':
                selected = F[bn.nanargmax(bn.nansum(fmm, axis=0))]
            elif self.method == 'JMIM':
                if bn.allnan(bn.nanmin(fmm, axis = 0)):
                    break
                selected = F[bn.nanargmax(bn.nanmin(fmm, axis=0))]
            elif self.method == 'MRMR':
                if bn.allnan(bn.nanmean(fmm, axis = 0)):
                    break
                MRMR = xy_MI[F] - bn.nanmean(fmm, axis=0)
                selected = F[bn.nanargmax(MRMR)]
                S_mi.append(bn.nanmax(MRMR))

            # record the JMIM of the newly selected feature and add it to S
            if self.method != 'MRMR':
                S_mi.append(bn.nanmax(bn.nanmin(fmm, axis=0)))
            S, F = self._add_remove(S, F, selected)

            # notify user
            if self.verbose > 0:
                self._print_results(S, S_mi)

            # if n_features == 'auto', let's check the S_mi to stop
            if self.n_features == 'auto' and len(S) > 10:
                # smooth the 1st derivative of the MI values of previously sel
                MI_dd = signal.savgol_filter(S_mi[1:], 9, 2, 1)
                # does the mean of the last 5 converge to 0?
                if np.abs(np.mean(MI_dd[-5:])) < 1e-3:
                    break

        # ---------------------------------------------------------------------
        # SAVE RESULTS
        # ---------------------------------------------------------------------

        self.n_features_ = len(S)
        self._support_mask = np.zeros(p, dtype=bool)
        self._support_mask[S] = True
        self.ranking_ = S
        self.mi_ = S_mi

        return self

    def _isinteger(self, x):
        return np.all(np.equal(np.mod(x, 1), 0))

    def _check_params(self, X, y):
        # checking input data and scaling it if y is continuous
        X, y = check_X_y(X, y)

        if not self.categorical:
            ss = StandardScaler()
            X = ss.fit_transform(X)
            y = ss.fit_transform(y.reshape(-1, 1))

        # sanity checks
        methods = ['JMI', 'JMIM', 'MRMR']
        if self.method not in methods:
            raise ValueError('Please choose one of the following methods:\n' +
                             '\n'.join(methods))

        if not isinstance(self.k, int):
            raise ValueError("k must be an integer.")
        if self.k < 1:
            raise ValueError('k must be larger than 0.')
        if self.categorical and np.any(self.k > np.bincount(y)):
            raise ValueError('k must be smaller than your smallest class.')

        if not isinstance(self.categorical, bool):
            raise ValueError('Categorical must be Boolean.')
        
        if not self.categorical and self._isinteger(y):
            print ('Are you sure y is continuous? It seems to be discrete.')
        if self._isinteger(X):
            print ('The values of X seem to be discrete. MI_FS will treat them'
                   'as continuous.')
        return X, y

    def _add_remove(self, S, F, i):
        """
        Helper function: removes ith element from F and adds it to S.
        """

        S.append(i)
        F.remove(i)
        return S, F

    def _print_results(self, S, MIs):
        out = ''
        if self.n_features == 'auto':
            out += 'Auto selected feature #' + str(len(S)) + ' : ' + str(S[-1])
        else:
            out += ('Selected feature #' + str(len(S)) + ' / ' +
                    str(self.n_features) + ' : ' + str(S[-1]))

        if self.verbose > 1:
            out += ', ' + self.method + ' : ' + str(MIs[-1])
        print (out)